In [ ]:
import scapy.all as scapy
import tkinter as tk
from tkinter import ttk
from tkinter import *
from tkinter.ttk import *
from tkinter import scrolledtext as st
import sys
from tkinter import filedialog as fd
from tkinter import messagebox as mb

class Monitor:
    
    def __init__(self, window):
        # Initializations
        self.ip = StringVar()
        self.mac = StringVar()
        self.wind = window
        self.wind.title('Network Monitor')
        
        self.labelIp = Label(self.wind, text='IP: ')
        self.labelIp.place(x=50, y=10)
        self.inputIp = Entry(self.wind, textvariable = self.ip)
        self.inputIp.place(x=100, y=10)
        
        self.labelMac = Label(self.wind, text='MAC: ')
        self.labelMac.place(x=50, y=40)
        self.inputIp = Entry(self.wind, textvariable = self.mac)
        self.inputIp.place(x=100, y=40)
        #self.scrolledtext1=st.ScrolledText(self.wind, width=100, height=10)
        #self.scrolledtext1.pack(fill=tk.BOTH, side=tk.LEFT, expand=True)
        self.scanned_output = self.scan('192.168.1.254/24')
        self.build_table_IP(self.scanned_output)
        
        
    def build_table_IP(self, result):
        self.table =ttk.Treeview(self.wind)
        self.table['columns']=('ip','mac')
        
        self.table.column('#0', width=0, stretch=NO)
        self.table.column('ip', anchor=CENTER, width=120)
        self.table.column('mac', anchor=CENTER, width=150)
        
        self.table.heading('#0',text = '', anchor = CENTER)
        self.table.heading('ip', text = 'Direcciòn IP', anchor = CENTER)
        self.table.heading('mac', text = 'Direcciòn MAC', anchor = CENTER)
        for i in self.result:
            self.table.insert(parent='', index=0, text='', values=(i["ip"],i["mac"]))
            
        self.table.bind("<Double- 1>", self.select_click_ip_mac)
        self.table.pack()
    
    def select_click_ip_mac(self, event):
        item = self.table.identify('item',event.x,event.y)
        self.ip.set(self.table.item(item,"values")[0])
        self.mac.set(self.table.item(item,"values")[1])
        
    def scan(self,ip):
        self.arp_req_frame = scapy.ARP(pdst = ip)
        self.broadcast_ether_frame = scapy.Ether(dst = "ff:ff:ff:ff:ff:ff")
    
        self.broadcast_ether_arp_req_frame = self.broadcast_ether_frame / self.arp_req_frame

        self.answered_list = scapy.srp(self.broadcast_ether_arp_req_frame, timeout = 1, verbose = False)[0]
        self.result = []
        for i in range(0,len(self.answered_list)):
            self.client_dict = {"ip" : self.answered_list[i][1].psrc, "mac" : self.answered_list[i][1].hwsrc}
            self.result.append(self.client_dict)

        return self.result
    
if __name__ == '__main__':
    window = tk.Tk()
    application = Monitor(window)
    window.mainloop()